In [3]:
!pip install simpletransformers
!pip install Cython

     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 1.0MB 8.8MB/s 
     |████████████████████████████████| 317kB 15.4MB/s 
     |████████████████████████████████| 1.6MB 16.2MB/s 
     |████████████████████████████████| 3.0MB 37.3MB/s 
     |████████████████████████████████| 7.2MB 42.4MB/s 
     |████████████████████████████████| 1.1MB 42.1MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 163kB 45.1MB/s 
     |████████████████████████████████| 122kB 39.5MB/s 
     |████████████████████████████████| 4.4MB 39.2MB/s 
     |████████████████████████████████| 112kB 45.5MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp

In [1]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'apex'...
remote: Enumerating objects: 7443, done.
remote: Total 7443 (delta 0), reused 0 (delta 0), pack-reused 7443
Receiving objects: 100% (7443/7443), 13.91 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (5028/5028), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-7q27g3rk
Created temporary directory: /tmp/pip-req-tracker-6spl7gvf
Created requirements tracker '/tmp/pip-req-tracker-6spl7gvf'
Created temporary directory: /tmp/pip-install-md_ddyhn
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-z56otttj
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-6spl7gvf'
    Running setup.py (path:/tmp/pip-req-build-z56otttj/setup.py) egg_info for package from file:///content/apex
    Runnin

In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from simpletransformers.classification import ClassificationModel 
 
from google.colab import drive
drive.mount('/content/drive')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Mounted at /content/drive


In [4]:
!ls drive/'My Drive'/RAYS_OF_HOPE/DATASET/



NewTest.csv  Test.csv  Test_Label.csv  Train.csv


In [6]:
dataset_path = 'drive/My Drive/RAYS_OF_HOPE/DATASET/'

In [7]:
train = pd.read_csv('{}/Train.csv'.format(dataset_path))
test = pd.read_csv('{}/Test.csv'.format(dataset_path))
test_labels = pd.read_csv('{}/Test_Label.csv'.format(dataset_path))

'''
test_labels['label'] = test_labels.sum(axis=1)
test['id']=test.id.astype(str)
test_labels['id']=test_labels.id.astype(str)
full_test=test.set_index('id').join(test_labels.set_index('id'),on = ['id'], how = 'inner')
full_test.head(5)
full_test = full_test[(full_test.label !=  -6)]
full_test['label'] = full_test.label.apply(lambda x:x if x==0 else 1)
list_to_add = list(train)
list_to_add.remove('id')
list_to_add.remove('comment_text')
train['label']=train[list_to_add].sum(axis =1)
train['label']= train.label.apply(lambda x:x if x==0 else 1)
'''

import re, string
re_tok = re.compile(f'([“”¨«»®´·º½¾¿¡§£₤‘’])')
def clean(s):
    s = re.sub(re_tok, '', s)
    return s.replace('\n', ' ')
train['sentences'] = train.sentences.apply(lambda x: clean(x))
test['sentences'] = test.sentences.apply(lambda x: clean(x))
train.head(5)

,sentences,classification_label
0,I am being harassed to pay a debt that is not ...,Policy violated
1,Information is not mine I have made several at...,Policy violated
2,I filled out the request form for my free cred...,Policy violated
3,i see that a loan was placed on my credit that...,Policy violated
4,Thank you Billy Bob.,Policy not violated


In [8]:
args = {'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5,'num_train_epochs':1, 'max_seq_length': 512,'overwrite_output_dir': True, 'save_steps':30000}
model = ClassificationModel('roberta', 'roberta-base', num_labels=6, args=args, use_cuda=True)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [9]:
train['classification_label'] = train.classification_label.astype('category').cat.codes
train.head(10)
train_df = pd.DataFrame(train)
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:303: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:

test_labels['classification_label'] = test_labels.classification_label.astype('category').cat.codes
tl_df = pd.DataFrame(test_labels)

from sklearn.metrics import f1_score, accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
result, model_outputs, wrong_predictions = model.eval_model(tl_df, f1=f1_multiclass, acc=accuracy_score)
print(result)


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:775: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.9625334218796219, 'f1': 0.9809523809523809, 'acc': 0.9809523809523809, 'eval_loss': 0.20845986130060973}
[[ 3.7315714   0.881753   -1.1142993  -1.187869   -0.84518427 -1.0635939 ]
 [ 3.6091697   2.611134   -1.383625   -1.39351    -1.5009944  -1.4200987 ]
 [ 2.285416    3.6097443  -1.4309239  -1.3059663  -1.5176636  -1.4323074 ]
 ...
 [ 3.7472286   0.8757202  -1.143458   -1.1851088  -0.8665477  -1.1042261 ]
 [ 3.5884063   0.68987846 -0.99830073 -1.057954   -0.79504454 -0.99251384]
 [ 2.4135926   3.496318   -1.3728254  -1.3836528  -1.5245341  -1.4221736 ]]


In [76]:
predictions, raw_outputs = model.predict(['I can honestly say that this is the worst mortgage experience I''ve ever had'])
print(raw_outputs)
print("_______________")
print(predictions)


[[ 2.5147386   2.4826114  -0.85065496 -1.4398165  -0.6137377  -0.5775348 ]]
_______________
[0]


In [78]:
import pickle


In [80]:
with open('roberta.sav','wb') as f:
     pickle.dump(model,f)

In [88]:
with open('roberta.pickle','rb') as f:
     val = pickle.load(f)
type(val)

simpletransformers.classification.classification_model.ClassificationModel

In [96]:
#predictions, raw_outputs = val.predict(['not good Morning'])
testing = ['good morning', 'information from a third-party reporting agency and therefore that information must be deleted','Highly professional','Give my money back','Bad service','not bad']
predictions, raw_outputs = val.predict(testing)
#ynew = model.predict()
print(predictions)
print("___________")



[0 1 0 1 1 0]
___________
